In [ ]:
import os
import shutil
import pandas as pd
import re
import cv2
import unicodedata
from tqdm import tqdm
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


라벨 리스트 만들기

In [ ]:
col=["Title", "Style","Gender", "Season"]

In [ ]:
df = pd.read_csv('1.csv', encoding = 'cp949', names = col)

In [ ]:
df['Title'][0]

'dandy_1_간편한 셋업 룩'

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/imgimg'

In [ ]:
mu_list = os.listdir(path)

In [ ]:
dic_label = {
    'male_list' : [],
    'female_list' : [],
    'spring_list' : [],
    'summer_list' : [],
    'winter_list' : [],
    'dandy_list' : [],
    'sports_list' : [],
    'casual_list' : []
}

In [ ]:
for i in range(len(df['Title'])):
  if df['Gender'][i] == 0:
    dic_label['male_list'].append(df['Title'][i])
  else:
    dic_label['female_list'].append(df['Title'][i])
  
  if df['Season'][i] == 0:
    dic_label['spring_list'].append(df['Title'][i])
  elif df['Season'][i] == 1:
    dic_label['summer_list'].append(df['Title'][i])
  else:
    dic_label['winter_list'].append(df['Title'][i])
  
  if df['Style'][i] == 0:
    dic_label['dandy_list'].append(df['Title'][i])
  elif df['Style'][i] == 1:
    dic_label['sports_list'].append(df['Title'][i])
  else:
    dic_label['casual_list'].append(df['Title'][i])



In [ ]:
len(dic_label['sports_list'])

1112

유니코드 맞춰주기

In [ ]:
uni = []
for i in mu_list:
  uni.append(unicodedata.normalize('NFC', i))


계절 폴더나누기

In [ ]:
!mkdir 계절
!mkdir 계절/봄가을
!mkdir 계절/여름
!mkdir 계절/겨울

In [ ]:
for i in tqdm( range(len(mu_list))):
  try:
    if uni[i].split('.')[0] in dic_label['spring_list']:
      lie = path + '/' + unicodedata.normalize('NFD', mu_list[i])
      shutil.copy(lie, '/content/계절/봄가을')
  except:
    print(i)
    pass

100%|██████████| 8551/8551 [02:49<00:00, 50.43it/s]  


In [ ]:
for i in tqdm( range(len(mu_list))):
  try:
    if uni[i].split('.')[0] in dic_label['summer_list']:
      lie = path + '/' + unicodedata.normalize('NFD', mu_list[i])
      shutil.copy(lie, '/content/계절/여름')
  except:
    print(i)
    pass

100%|██████████| 8551/8551 [00:03<00:00, 2673.05it/s]


In [ ]:
for i in tqdm( range(len(mu_list))):
  try:
    if uni[i].split('.')[0] in dic_label['winter_list']:
      lie = path + '/' + unicodedata.normalize('NFD', mu_list[i])
      shutil.copy(lie, '/content/계절/겨울')
  except:
    print(i)
    pass

100%|██████████| 8551/8551 [00:02<00:00, 3074.60it/s]


In [ ]:
path_season = '/content/계절'
label_season = os.listdir(path_season)

In [ ]:
dataset_season = {}
for i in os.listdir(path_season):
  sub_path = path_season + '/' + i + '/'
  dataset_season[i] = []
  for j in os.listdir(sub_path):
    dataset_season[i].append(sub_path+j)

In [ ]:
len(dataset_season)

3

resize 전처리

In [ ]:
!mkdir resized
!mkdir resized/봄가을
!mkdir resized/여름
!mkdir resized/겨울

In [ ]:
for label, filenames in dataset_season.items():
    for filename in filenames:
        img = cv2.imread(filename)

        # 이미지의 x, y가 224이 넘을 경우 작게해주기
        percent = 1
        if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세보다 크면 가로를 640으로 맞추고 세로를 비율에 맞춰서
            percent = 224/img.shape[1]
        else :
            percent = 224/img.shape[0]

        img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                # 이미지 범위 지정
        y,x,h,w = (0,0,img.shape[0], img.shape[1])

        # 그림 주변에 검은색으로 칠하기
        w_x = (224-(w-x))/2  # w_x = (224 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
        h_y = (224-(h-y))/2

        if(w_x < 0):         # 크기가 -면 0으로 지정.
            w_x = 0
        elif(h_y < 0):
            h_y = 0

        M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
       
        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
        cv2.imwrite('/content/resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)

In [ ]:
path2 = '/content/resized'

In [ ]:
dataset2 = {}
for i in os.listdir(path2):
  sub_path = path2 + '/' + i + '/'
  dataset2[i] = []
  for j in os.listdir(sub_path):
    dataset2[i].append(sub_path+j)

In [ ]:
len(dataset2['겨울'])

1393

skin 전처리 : 살색을 제외하고 검정색으로 처리


In [ ]:
!mkdir skin
!mkdir skin/봄가을
!mkdir skin/여름
!mkdir skin/겨울


In [ ]:
skin_color = (255, 195, 150)
skin_min = (0, 48, 80)
skin_max = (20, 255, 255)

In [ ]:
for label, filenames in dataset2.items():
    for filename in filenames:
        img = cv2.imread(filename, cv2.IMREAD_COLOR)
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_mask = cv2.inRange(img_hsv, skin_min, skin_max)
        img_skin = cv2.bitwise_and(img, img, mask = img_mask)



        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
        cv2.imwrite('/content/skin/{0}/{1}'.format(label, filename.split("/")[-1]) , img_skin)

In [ ]:
path3 = '/content/skin'

In [ ]:
dataset3 = {}
for i in os.listdir(path3):
  sub_path = path3 + '/' + i + '/'
  dataset3[i] = []
  for j in os.listdir(sub_path):
    dataset3[i].append(sub_path+j)